In [3]:
import Trade

In [6]:
AAL_long_order = Trade.go_long('AAL',100,100,120,80)

Paper API connected
Go long order submitted:
Order ID: b043c257-8550-4872-bf17-996bb9d59958
Entry point: 100
Take profit point: 120
Stop loss point: 79.2


In [ ]:
Trade.api_access()